In [ ]:
from enums import CollectionNames, EmbeddingModels
from qdrant_client.models import Filter, FieldCondition, MatchValue

from retrieval_util import (
    get_qdrant_client,
    get_embeddings_function,
    retrieve_run_dict_with_documents_with_scores,
    create_llm_for_self_query_retriever
)

import os
import json
import logging
from ranx import Qrels, Run, evaluate, compare

os.environ[
    "TOKENIZERS_PARALLELISM"] = "false"  # Verhindert Fehler beim Erzeugen von Embeddings (Tritt sporadisch auf, konnte ich nicht gezielt reproduzieren)
logging.basicConfig(level=logging.INFO)

**Dieses Notebook liefert die Ergebnisse, die auch in der Arbeit für das intfloat/multilingual-e5-large aufgegriffen und bewertet werden.**

**use_original_query=False** gesetzt? -> In retrieval_util.py setzen! Siehe kommentar im unteren Feld

Nach Änderung muss der Jupyter Server neu gestartet werden, damit die Änderungen wirksam werden.


**Ergebnisse wurden @5 und @10 bestimmt. Es werden nur Ergebnisse @10 betrachtet, da das der Maßstab im MTEB ist.**

In [3]:
# in der retrieval_util.py muss der Parameter use_original_query=False gesetzt werden (in der funktion für den SelfQueryRetriever)
# Behilfslösung, da ich nicht die Signatur der Funktion umschreiben will, sonst muss das beim Aufruf in den vorherigen Evaluationen auch wieder angepasst werden
SELF_QUERY_RETRIEVER_FRAGEN = {
    1: "Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?",
    7: "Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?",
    10: "Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?"
}
'''
Richtig vergleichbar ist dieser Ansatz nicht, da die Fragen nicht identisch sind.
Ebenso werden die vereinfachten Fragen nicht mit den selbst formulierten übereinstimmen, da sie durch das LLM generiert werden.

Hiermit soll dennoch gezeigt werden, wie dieser Ansatz performt.
Für ein reales System eignet sich der SelfQueryRetriever potenziell nicht.
Wenn in einem erweiterten System weitere Felder für das Filtern (z. B. Studienmodelle) hinzugefügt werden,
müssten auch diese in der Fragestellung eingebunden werden. Die Fragen würden immer länger und komplexer werden.
Dieser Aspekt ist jedoch nicht Teil der Arbeit.

Daher wird der Ansatz zwar mit evaluiert, in der Arbeit schließe ich diesen jedoch begründet aus.
'''

'\nRichtig vergleichbar ist dieser Ansatz nicht, da die Fragen nicht identisch sind.\nEbenso werden die vereinfachten Fragen nicht mit den selbst formulierten übereinstimmen, da sie durch das LLM generiert werden.\n\nHiermit soll dennoch gezeigt werden, wie dieser Ansatz performt.\nFür ein reales System eignet sich der SelfQueryRetriever potenziell nicht.\nWenn in einem erweiterten System weitere Felder für das Filtern (z. B. Studienmodelle) hinzugefügt werden,\nmüssten auch diese in der Fragestellung eingebunden werden. Die Fragen würden immer länger und komplexer werden.\nDas ist aber ein Aspekt der Gebrauchstauglichkeit und nicht der Funktionsweise/Performance des Systems.\n\nDaher wird der Ansatz zwar mit evaluiert, in der Arbeit schließe ich diesen jedoch aufgrund der genannten Aspekte aus.\n'

In [4]:
# Konstanten
MAX_DOCUMENTS = 10
BASE_PATH = "./infloat_retrieval_results_AT_10"
LLM = create_llm_for_self_query_retriever()
FRAGEN = {
    1: "Wann kann ich mich einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit schreiben?",
    7: "Wie sind Portfolioprüfungen aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit?",
    10: "Wie lange dauern die Klausuren?"
}
METRICS = ["ndcg@10", "mrr@10", "map@10", "precision@10", "recall@10", "hits@10", "hit_rate@10"]

In [5]:
# Alle relevanten Filter für die Fragen
def create_field_condition(field, value):
    return FieldCondition(
        key=f"metadata.{field}",
        match=MatchValue(value=value)
    )


FILTERS = {
    1: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Meschede"),
        create_field_condition("abschluss", "Master")
    ]),
    2: Filter(must=[
        create_field_condition("standort", "Soest"),
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    3: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    4: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    5: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    6: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    7: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Soest"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    8: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    9: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    10: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
}

In [6]:
def save_results_to_file(file_path, results_dict):
    try:
        with open(file_path, "w") as file:
            json.dump(results_dict, file, indent=4)
        print(f"Ergebnisse wurden erfolgreich in {file_path} gespeichert.")
    except Exception as e:
        print(f"Fehler beim Speichern der Datei: {e}")

In [7]:
# Mapping (qrels) für die Collections mit fester Länge
token_based_qrels_dict = {
    "q_1": {
        "d_1021": 10,
        "d_1022": 1
    },
    "q_2": {
        "d_1301": 10
    },
    "q_3": {
        "d_1476": 10,
        "d_1480": 10,
        "d_1481": 2
    },
    "q_4": {
        "d_1498": 10,
        "d_1499": 10,
        "d_1500": 2
    },
    "q_5": {
        "d_1482": 10,
        "d_1483": 10,
        "d_1484": 10,
        "d_1485": 10
    },
    "q_6": {
        "d_18": 10
    },
    "q_7": {
        "d_1192": 10,
        "d_1193": 8
    },
    "q_8": {
        "d_1530": 10,
        "d_1553": 10,
        "d_1554": 10,
        "d_1555": 10
    },
    "q_9": {
        "d_704": 10
    },
    "q_10": {
        "d_14": 10,
        "d_15": 10
    }
}

TOKEN_BASED_QRELS = Qrels(token_based_qrels_dict)

In [8]:
# Mapping (qrels) für die Collections mit rekursiv gebildeten Chunks
recursive_qrels_dict = {
    "q_1": {
        "d_1371": 10,
        "d_1370": 1
    },
    "q_2": {
        "d_1866": 10
    },
    "q_3": {
        "d_2292": 10,
        "d_2293": 2,
        "d_2286": 10
    },
    "q_4": {
        "d_2313": 10,
        "d_2314": 10,
        "d_2315": 2
    },
    "q_5": {
        "d_2290": 4,
        "d_2295": 10,
        "d_2296": 10,
        "d_2297": 10,
        "d_2298": 10
    },
    "q_6": {
        "d_25": 10
    },
    "q_7": {
        "d_1586": 10
    },
    "q_8": {
        "d_2388": 10,
        "d_2389": 10,
        "d_2390": 10,
        "d_2356": 8,
        "d_2357": 8
    },
    "q_9": {
        "d_946": 10
    },
    "q_10": {
        "d_19": 10
    }
}

RECURSIVE_QRELS = Qrels(recursive_qrels_dict)

In [9]:
embeddings = get_embeddings_function(model_name=EmbeddingModels.INFLOAT.value)

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-16 13:27:40,691 [INFO] PyTorch version 2.6.0 available.
2025-03-16 13:27:40,923 [INFO] Use pytorch device_name: mps
2025-03-16 13:27:40,923 [INFO] Load pretrained SentenceTransformer: intfloat/multilingual-e5-large


In [10]:
qdrant_url = "http://localhost:6333"
token_based_client = get_qdrant_client(embeddings=embeddings, collection_name=CollectionNames.INFLOAT_TOKEN_BASED.value,
                                       qdrant_url=qdrant_url)
recursive_client = get_qdrant_client(embeddings=embeddings, collection_name=CollectionNames.INFLOAT_RECURSIVE.value,
                                     qdrant_url=qdrant_url)

2025-03-16 13:27:46,309 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-16 13:27:46,319 [INFO] HTTP Request: GET http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks "HTTP/1.1 200 OK"
2025-03-16 13:27:46,624 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-16 13:27:46,629 [INFO] HTTP Request: GET http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks "HTTP/1.1 200 OK"


In [11]:
# Run dicts ohne Filter, mit optimalen Filtern und mit Self-Query-Retriever erzeugen
token_based_result_no_filters = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                             fragen_dict=FRAGEN, mode="no_filters",
                                                                             max_documents=MAX_DOCUMENTS)

token_based_result_optimal_filters = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                                  fragen_dict=FRAGEN,
                                                                                  mode="optimal_filters",
                                                                                  filter_dict=FILTERS,
                                                                                  max_documents=MAX_DOCUMENTS)

token_based_result_self_query_retriever = retrieve_run_dict_with_documents_with_scores(client=token_based_client,
                                                                                       fragen_dict=SELF_QUERY_RETRIEVER_FRAGEN,
                                                                                       mode="self_query_retriever",
                                                                                       llm=LLM,
                                                                                       search_kwargs={
                                                                                           "k": MAX_DOCUMENTS})

2025-03-16 13:27:46,786 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:27:46,907 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:27:47,009 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:27:47,050 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:27:47,109 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:27:47,147 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_token_based_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16

# Token-Basierte Collections

In [12]:
# Run dicts in Dateien schreiben, um sie einsehen zu können
save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_no_filters.json",
                     results_dict=token_based_result_no_filters)

save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_optimal_filters.json",
                     results_dict=token_based_result_optimal_filters)

save_results_to_file(file_path=f"{BASE_PATH}/token_based_result_self_query_retriever.json",
                     results_dict=token_based_result_self_query_retriever)

Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/token_based_result_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/token_based_result_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/token_based_result_self_query_retriever.json gespeichert.


In [13]:
# Runs erzeugen
run_token_based_result_no_filters = Run(token_based_result_no_filters, name="token_based_result_no_filters")

run_token_based_result_optimal_filters = Run(token_based_result_optimal_filters,
                                             name="token_based_result_optimal_filters")

run_token_based_result_self_query_retriever = Run(token_based_result_self_query_retriever,
                                                  name="token_based_result_self_query_retriever")

In [14]:
# Evaluation durchführen
token_based_scores_no_filters = evaluate(qrels=TOKEN_BASED_QRELS, run=run_token_based_result_no_filters,
                                         metrics=METRICS)

token_based_scores_optimal_filters = evaluate(qrels=TOKEN_BASED_QRELS, run=run_token_based_result_optimal_filters,
                                              metrics=METRICS)

token_based_scores_self_query_retriever = evaluate(qrels=TOKEN_BASED_QRELS,
                                                   run=run_token_based_result_self_query_retriever, metrics=METRICS)

In [15]:
# Scores in Dateien schreiben, um sie einsehen zu können
save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_no_filters.json",
                     results_dict=token_based_scores_no_filters)

save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_optimal_filters.json",
                     results_dict=token_based_scores_optimal_filters)

save_results_to_file(file_path=f"{BASE_PATH}/token_based_scores_self_query_retriever.json",
                     results_dict=token_based_scores_self_query_retriever)

Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/token_based_scores_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/token_based_scores_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/token_based_scores_self_query_retriever.json gespeichert.


In [16]:
report = compare(
    qrels=TOKEN_BASED_QRELS,
    runs=[run_token_based_result_no_filters, run_token_based_result_optimal_filters, run_token_based_result_self_query_retriever],
    metrics=METRICS
)

In [17]:
print(report)

#    Model                                    NDCG@10      MRR@10  MAP@10      P@10    Recall@10    Hits@10    Hit Rate@10
---  ---------------------------------------  ---------  --------  --------  ------  -----------  ---------  -------------
a    token_based_result_no_filters            0.346         0.389  0.273       0.14        0.533        1.4            0.7
b    token_based_result_optimal_filters       0.720ᵃ        0.8    0.663ᵃ      0.21        0.8          2.1            0.8
c    token_based_result_self_query_retriever  0.382         0.429  0.350       0.13        0.483        1.3            0.6


In [18]:
with open(f"{BASE_PATH}/token_report@{MAX_DOCUMENTS}.txt", "w") as file:
    file.write(str(report))

# Rekursive Collections

In [19]:
recursive_result_no_filters = retrieve_run_dict_with_documents_with_scores(client=recursive_client, fragen_dict=FRAGEN,
                                                                           mode="no_filters",
                                                                           max_documents=MAX_DOCUMENTS)

recursive_result_optimal_filters = retrieve_run_dict_with_documents_with_scores(client=recursive_client,
                                                                                fragen_dict=FRAGEN,
                                                                                mode="optimal_filters",
                                                                                filter_dict=FILTERS,
                                                                                max_documents=MAX_DOCUMENTS)

recursive_result_self_query_retriever = retrieve_run_dict_with_documents_with_scores(client=recursive_client,
                                                                                     fragen_dict=SELF_QUERY_RETRIEVER_FRAGEN,
                                                                                     mode="self_query_retriever",
                                                                                     llm=LLM,
                                                                                     search_kwargs={"k": MAX_DOCUMENTS})

2025-03-16 13:29:15,873 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:29:15,926 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:29:15,975 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:29:16,017 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:29:16,086 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:29:16,133 [INFO] HTTP Request: POST http://localhost:6333/collections/intfloat_multilingual-e5-large_recursive_chunks/points/query "HTTP/1.1 200 OK"
2025-03-16 13:29:16,20

In [20]:
save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_no_filters.json",
                     results_dict=recursive_result_no_filters)

save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_optimal_filters.json",
                     results_dict=recursive_result_optimal_filters)

save_results_to_file(file_path=f"{BASE_PATH}/recursive_result_self_query_retriever.json",
                     results_dict=recursive_result_self_query_retriever)

Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/recursive_result_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/recursive_result_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/recursive_result_self_query_retriever.json gespeichert.


In [21]:
run_recursive_result_no_filters = Run(recursive_result_no_filters, name="recursive_result_no_filters")

run_recursive_result_optimal_filters = Run(recursive_result_optimal_filters, name="recursive_result_optimal_filters")

run_recursive_result_self_query_retriever = Run(recursive_result_self_query_retriever, name="recursive_result_self_query_retriever")

In [22]:
recursive_scores_no_filters = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_no_filters, metrics=METRICS)

recursive_scores_optimal_filters = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_optimal_filters,
                                            metrics=METRICS)

recursive_scores_self_query_retriever = evaluate(qrels=RECURSIVE_QRELS, run=run_recursive_result_self_query_retriever, metrics=METRICS)

In [23]:
save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_no_filters.json",
                     results_dict=recursive_scores_no_filters)

save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_optimal_filters.json",
                     results_dict=recursive_scores_optimal_filters)

save_results_to_file(file_path=f"{BASE_PATH}/recursive_scores_self_query_retriever.json",
                     results_dict=recursive_scores_self_query_retriever)

Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/recursive_scores_no_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/recursive_scores_optimal_filters.json gespeichert.
Ergebnisse wurden erfolgreich in ./infloat_retrieval_results_AT_10/recursive_scores_self_query_retriever.json gespeichert.


In [24]:
report = compare(
    qrels=RECURSIVE_QRELS,
    runs=[run_recursive_result_no_filters, run_recursive_result_optimal_filters, run_recursive_result_self_query_retriever],
    metrics=METRICS
)

In [25]:
print(report)

#    Model                                  NDCG@10    MRR@10    MAP@10      P@10    Recall@10    Hits@10    Hit Rate@10
---  -------------------------------------  ---------  --------  --------  ------  -----------  ---------  -------------
a    recursive_result_no_filters            0.464      0.458     0.367       0.18        0.697        1.8            0.8
b    recursive_result_optimal_filters       0.827ᵃᶜ    0.900ᵃ    0.808ᵃᶜ     0.22        0.9          2.2            0.9
c    recursive_result_self_query_retriever  0.428      0.517     0.393       0.15        0.587        1.5            0.7


In [26]:
with open(f"{BASE_PATH}/recursive_report@{MAX_DOCUMENTS}.txt", "w") as file:
    file.write(str(report))